In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Overview

### Protecting Gen AI Applications from Data Leakage
Large language models (LLMs) are deep learning models trained on massive datasets of text. LLMs can translate language, summarize text, generate creative writing, generate code, power chatbots and virtual assistants, and complement search engines and recommendation systems.


When incorporating your own data into generative AI applications especially via [Model Tuning](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models) it's possible to return a prediction with data from your dataset. In this case many organizations may want to filter LLM responses for sensitive data.


By using Sensitive Data Protection(Cloud DLP) we can identify and take corrective action on sensitive data within LLM responses in real time.


### Using Sensitive Data Protection(Cloud DLP) to filter LLM Responses

[Sensitive Data Protection(Cloud DLP)](https://cloud.google.com/dlp) is a fully managed service designed to discover, classify, and protect your sensitive data, where it resides from databases, text-based content, or even images. It uses a variety of methods to identify sensitive data, including regular expressions, dictionaries, and contextual elements. Once sensitive data is identified, Sensitive Data Protection(Cloud DLP) can take several actions to either classify it, mask it, encrypt it or even delete it.


Sensitive Data Protection(Cloud DLP) can be accessed via Cloud Console and used to scan data within Cloud Storage, BigQuery and other Google Cloud services. The following notebook demonstrates using it through the SDK to incorporate Sensitive Data Protection(Cloud DLP) capabilities directly into you Generative AI enabled applications


### Objectives

In this tutorial, you will learn how to use Sensitive Data Protection(Cloud DLP) API with the Python SDK and explore how to identify and redact sensitive data within a response from PaLM 2 LLM

By the end of the notebook, you should be able to understand various configurations of Sensitive Data Protection(Cloud DLP) like `inspect_config`, `deidentify_config`, `item`, and what each variable controls.

The steps performed include:

- Installing the Python SDKs
- Understand a Data Leakage scenario
  - Text generation model with `text-bison@001`
    - Understanding prompt manipulation to return sensitive data
- Understand Data Leakage Mitigations
  - Using Sensitive Data Protection(Cloud DLP) with `text-bison@001` responses
  

### Costs
This tutorial uses billable components of Google Cloud:

* Vertex AI Generative AI Studio
* Sensitive Data Protection(Cloud DLP)

Learn about pricing for [Vertex AI](https://cloud.google.com/vertex-ai/pricing), and
 [Sensitive Data Protection(Cloud DLP)](https://cloud.google.com/dlp/pricing). Use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Data governance and security
For more information, see the documentation on [Data Governance and Generative AI](https://cloud.google.com/vertex-ai/docs/generative-ai/data-governance) on Google Cloud.

### Responsible AI
Large language models (LLMs) can translate language, summarize text, generate creative writing, generate code, power chatbots and virtual assistants, and complement search engines and recommendation systems. At the same time, as an early-stage technology, its evolving capabilities and uses create potential for misapplication, misuse, and unintended or unforeseen consequences. Large language models can generate output that you don't expect, including text that's offensive, insensitive, or factually incorrect.

What's more, the incredible versatility of LLMs is also what makes it difficult to predict exactly what kinds of unintended or unforeseen outputs they might produce. Given these risks and complexities, the PaLM API is designed with [Google's AI Principles](https://ai.google/principles/) in mind. However, it is important for developers to understand and test their models to deploy safely and responsibly. To aid developers, the Generative AI Studio has built-in content filtering, and the PaLM API has safety attribute scoring to help customers test Google's safety filters and define confidence thresholds that are right for their use case and business. Please refer to the [Safety filters and attributes](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/responsible-ai#safety_filters_and_attributes) section to learn more.

When the PaLM API is integrated into a customer's unique use case and context, additional responsible AI considerations and [PaLM limitations](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/responsible-ai#palm_limitations) may need to be considered. We encourage customers to leverage fairness, interpretability, privacy and security [recommended practices](https://ai.google/responsibilities/responsible-ai-practices/).

## Getting Started

In [ ]:
#@title Install Vertex AI and Sensitive Data Protection SDK
# Install Google Cloud Vertex AI
!pip install google-cloud-aiplatform --upgrade --user
# Install DLP
! pip install google-cloud-dlp



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.5/157.5 kB 4.0 MB/s eta 0:00:00


###Restart Runtime
After installing the necessary Python SDKs you must restart the python runtime. There are a few options based on environment:

**Colab**:
1. Click the "Restart Runtime" button in the output of the SDK installs
2. Click "Runtime" on the top toolbar -> Click "Restart Runtime"
3. Run Colab Runtime Restart Code Block

**Vertex AI Workbench**:
1. Click "Kernel" on the top toolbar -> Click "Restart Kernel"

In [ ]:
#@title Colab Runtime Restart
#import os
#os.kill(os.getpid(), 9)

In [ ]:
#@title Set Project and Location
PROJECT_ID = "cloud-llm-preview1"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### Authenticating your notebook environment
* If you are using **Colab** to run this notebook, uncomment the cell below and continue.
* If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
from google.colab import auth
auth.authenticate_user(project_id=PROJECT_ID)


### Set Model for Prediction/Generation (text-bison@001)

In [ ]:
from vertexai.preview.language_models import (TextGenerationModel)

generation_model = TextGenerationModel.from_pretrained("text-bison@001")

## Understanding and Mitigating Data Leakage

In [ ]:
#@title Threat Use Case: Extract Sensitive Data & Data Leakage Scenario

prompt = f"""Who is the CEO of Google? What is their email?
  """
response = generation_model.predict(prompt)
response

Sundar Pichai is the CEO of Google. His email address is sundar@google.com.

In [ ]:
#@title Inspect and Redact PaLM 2 output with Sensitive Data Protection(Cloud DLP)

import google.cloud.dlp  # noqa: F811, E402
from typing import List  # noqa: F811, E402

def deidentify_with_replace_infotype(
    project: str, item: str, info_types: List[str]
) -> None:
    """Uses the Data Loss Prevention API to deidentify sensitive data in a
    string by replacing it with the info type.
    Args:
        project: The Google Cloud project id to use as a parent resource.
        item: The string to deidentify (will be treated as text).
        info_types: A list of strings representing info types to look for.
            A full list of info type categories can be fetched from the API.
    Returns:
        None; the response from the API is printed to the terminal.
    """

    # Instantiate a client
    dlp = google.cloud.dlp_v2.DlpServiceClient()

    # Convert the project id into a full resource id.
    parent = f"projects/{PROJECT_ID}"

    # Construct inspect configuration dictionary
    inspect_config = {"info_types": [{"name": info_type} for info_type in info_types]}

    # Construct deidentify configuration dictionary
    deidentify_config = {
        "info_type_transformations": {
            "transformations": [
                {"primitive_transformation": {"replace_with_info_type_config": {}}}
            ]
        }
    }

    # Call the API
    response = dlp.deidentify_content(
        request={
            "parent": parent,
            "deidentify_config": deidentify_config,
            "inspect_config": inspect_config,
            "item": {"value": item},
        }
    )

    # Print out the results.
    print(response.item.value)

deidentify_with_replace_infotype(PROJECT_ID, response.text, ["PERSON_NAME","EMAIL_ADDRESS"])

[PERSON_NAME] is the CEO of Google. His email address is [EMAIL_ADDRESS].
